In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder

In [42]:
Mdata=pd.read_csv('../data/mushrooms.csv')
Mdata.info(), Mdata.columns #13개 컬럼

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

(None,
 Index(['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
        'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
        'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
        'stalk-surface-below-ring', 'stalk-color-above-ring',
        'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
        'ring-type', 'spore-print-color', 'population', 'habitat'],
       dtype='object'))

In [43]:
M= Mdata['stalk-root']=='?'
remove= Mdata[M]['stalk-root'].index
Mdata.drop(index=remove, inplace=True)

In [44]:
type(Mdata['cap-color'].to_numpy())

numpy.ndarray

In [45]:
# 데이터가 순서가 상관없는 범주형 데이터-> 인코딩(피쳐가 많으므로 label_encoding)
# 피쳐는 나머지, 타겟은 class
Mdata['class']=Mdata['class'].replace({'p': 0, 'e':1})
Lencoder=LabelEncoder()
M_data=pd.DataFrame()
M_data['class']=Mdata['class']
feature_col=['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
        'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
        'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
        'stalk-surface-below-ring', 'stalk-color-above-ring',
        'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
        'ring-type', 'spore-print-color', 'population', 'habitat']
for feature in feature_col:
    encoder=LabelEncoder()
    en_feature=encoder.fit_transform(Mdata[feature])
    M_data[feature]=en_feature

M_data.info(), M_data.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 5644 entries, 0 to 8114
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   class                     5644 non-null   int64
 1   cap-shape                 5644 non-null   int32
 2   cap-surface               5644 non-null   int32
 3   cap-color                 5644 non-null   int32
 4   bruises                   5644 non-null   int32
 5   odor                      5644 non-null   int32
 6   gill-attachment           5644 non-null   int32
 7   gill-spacing              5644 non-null   int32
 8   gill-size                 5644 non-null   int32
 9   gill-color                5644 non-null   int32
 10  stalk-shape               5644 non-null   int32
 11  stalk-root                5644 non-null   int32
 12  stalk-surface-above-ring  5644 non-null   int32
 13  stalk-surface-below-ring  5644 non-null   int32
 14  stalk-color-above-ring    5644 non-null   int

(None,
    class  cap-shape  cap-surface  cap-color  bruises  odor  gill-attachment  \
 0      0          5            2          4        1     6                1   
 1      1          5            2          7        1     0                1   
 2      1          0            2          6        1     3                1   
 
    gill-spacing  gill-size  gill-color  ...  stalk-surface-below-ring  \
 0             0          1           2  ...                         2   
 1             0          0           2  ...                         2   
 2             0          0           3  ...                         2   
 
    stalk-color-above-ring  stalk-color-below-ring  veil-type  veil-color  \
 0                       5                       5          0           0   
 1                       5                       5          0           0   
 2                       5                       5          0           0   
 
    ring-number  ring-type  spore-print-color  population  habi

In [46]:
item=[]

for feature in Mdata.columns:
    i=Mdata[feature].unique()
    print(feature, i)
    item.extend(i)

print(item)



class [0 1]
cap-shape ['x' 'b' 's' 'f' 'k' 'c']
cap-surface ['s' 'y' 'f' 'g']
cap-color ['n' 'y' 'w' 'g' 'e' 'p' 'b' 'c']
bruises ['t' 'f']
odor ['p' 'a' 'l' 'n' 'f' 'c' 'm']
gill-attachment ['f' 'a']
gill-spacing ['c' 'w']
gill-size ['n' 'b']
gill-color ['k' 'n' 'g' 'p' 'w' 'h' 'u' 'r' 'y']
stalk-shape ['e' 't']
stalk-root ['e' 'c' 'b' 'r']
stalk-surface-above-ring ['s' 'f' 'k' 'y']
stalk-surface-below-ring ['s' 'f' 'y' 'k']
stalk-color-above-ring ['w' 'g' 'p' 'n' 'b' 'c' 'y']
stalk-color-below-ring ['w' 'p' 'g' 'b' 'n' 'c' 'y']
veil-type ['p']
veil-color ['w' 'y']
ring-number ['o' 't' 'n']
ring-type ['p' 'e' 'l' 'n']
spore-print-color ['k' 'n' 'u' 'h' 'r' 'w']
population ['s' 'n' 'a' 'v' 'y' 'c']
habitat ['u' 'g' 'm' 'd' 'p' 'l']
[0, 1, 'x', 'b', 's', 'f', 'k', 'c', 's', 'y', 'f', 'g', 'n', 'y', 'w', 'g', 'e', 'p', 'b', 'c', 't', 'f', 'p', 'a', 'l', 'n', 'f', 'c', 'm', 'f', 'a', 'c', 'w', 'n', 'b', 'k', 'n', 'g', 'p', 'w', 'h', 'u', 'r', 'y', 'e', 't', 'e', 'c', 'b', 'r', 's', 'f', '

In [47]:
# 분리
X_train, X_test, Y_train, Y_test= train_test_split(M_data[feature_col], M_data['class'], 
                                                   test_size=.3, stratify=M_data['class'])
print(f'X_train: {X_train.shape},{X_train.ndim}\n\
X_test: {X_test.shape},{X_test.ndim}\n\
Y_train: {Y_train.shape},{Y_train.ndim}\n\
Y_test: {Y_test.shape},{Y_test.ndim}')

X_train: (3950, 22),2
X_test: (1694, 22),2
Y_train: (3950,),1
Y_test: (1694,),1


In [48]:
log=LogisticRegression(solver='liblinear')
log.fit(X_train, Y_train)

LogisticRegression(solver='liblinear')

In [49]:
log.score(X_test, Y_test)

0.9645808736717828

In [50]:
# 모델 [1]-> 로지스틱스 회귀 with gridsearchCV
# params->'C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 
#           'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'
cv_model= GridSearchCV(estimator=LogisticRegression(solver='liblinear'), param_grid={'C':[1,2,3,5,10]}, 
                       return_train_score=True,
                       verbose=True, scoring=['r2'], cv=5, refit='r2')
cv_model.fit(X_train, Y_train)
result=pd.DataFrame(cv_model.cv_results_)
result
best_lr=cv_model.best_estimator_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


In [51]:
print(best_lr.score(X_test, Y_test))
best_lr.score(X_train,Y_train)

0.9917355371900827


0.9908860759493671

In [52]:
# 모델[2] ->결정나무
# 인코딩 없이 진행
# 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 
# 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter'

dt_model=GridSearchCV(estimator=DecisionTreeClassifier(), param_grid={"min_samples_split":[2,3,4,5,10], 'max_depth': [3,5,7,9,10,15]}, return_train_score=True, 
                      verbose=True,
                      cv=5, refit=True
                      )
dt_model.fit(X_train, Y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [3, 5, 7, 9, 10, 15],
                         'min_samples_split': [2, 3, 4, 5, 10]},
             return_train_score=True, verbose=True)

In [53]:
dt_result= pd.DataFrame(dt_model.cv_results_)
dt_result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.003228,0.000687,0.001614,0.000808,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.970886,0.94557,0.963291,...,0.961519,0.008533,26,0.963924,0.955063,0.965823,0.966139,0.965190,0.963228,0.004152
1,0.002284,0.001923,0.000699,0.000589,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.970886,0.94557,0.963291,...,0.961519,0.008533,26,0.963924,0.955063,0.965823,0.966139,0.965190,0.963228,0.004152
2,0.001589,0.001362,0.003489,0.005522,3,4,"{'max_depth': 3, 'min_samples_split': 4}",0.970886,0.94557,0.963291,...,0.961519,0.008533,26,0.963924,0.955063,0.965823,0.966139,0.965190,0.963228,0.004152
3,0.007119,0.007617,0.000864,0.000774,3,5,"{'max_depth': 3, 'min_samples_split': 5}",0.970886,0.94557,0.963291,...,0.961519,0.008533,26,0.963924,0.955063,0.965823,0.966139,0.965190,0.963228,0.004152
4,0.001030,0.001280,0.000397,0.000794,3,10,"{'max_depth': 3, 'min_samples_split': 10}",0.970886,0.94557,0.963291,...,0.961519,0.008533,26,0.963924,0.955063,0.965823,0.966139,0.965190,0.963228,0.004152
5,0.004643,0.005363,0.000754,0.000739,5,2,"{'max_depth': 5, 'min_samples_split': 2}",1.000000,1.00000,0.998734,...,0.998987,0.000947,21,0.998418,1.000000,0.998734,0.999051,0.998734,0.998987,0.000544
6,0.005044,0.006759,0.003757,0.005803,5,3,"{'max_depth': 5, 'min_samples_split': 3}",1.000000,1.00000,0.998734,...,0.998987,0.000947,21,0.998418,1.000000,0.998734,0.999051,0.998734,0.998987,0.000544
7,0.002047,0.001709,0.003967,0.006478,5,4,"{'max_depth': 5, 'min_samples_split': 4}",1.000000,1.00000,0.998734,...,0.998987,0.000947,21,0.998418,1.000000,0.998734,0.999051,0.998734,0.998987,0.000544
8,0.004735,0.005572,0.000927,0.000822,5,5,"{'max_depth': 5, 'min_samples_split': 5}",1.000000,1.00000,0.998734,...,0.998987,0.000947,21,0.998418,1.000000,0.998734,0.999051,0.998734,0.998987,0.000544
9,0.002394,0.001495,0.000600,0.000802,5,10,"{'max_depth': 5, 'min_samples_split': 10}",1.000000,1.00000,0.998734,...,0.998987,0.000947,21,0.998418,1.000000,0.998734,0.999051,0.998734,0.998987,0.000544


In [69]:
print(best_dt.feature_names_in_[[3,7,9,12,19,20,21,]])
best_dt.feature_importances_

['bruises' 'gill-size' 'stalk-shape' 'stalk-surface-below-ring'
 'spore-print-color' 'population' 'habitat']


array([0.        , 0.        , 0.        , 0.04753316, 0.        ,
       0.        , 0.        , 0.16403344, 0.        , 0.03539124,
       0.        , 0.        , 0.00511584, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.69285911,
       0.00591942, 0.04914779])

In [54]:
best_dt= dt_model.best_estimator_
print(best_dt.score(X_test, Y_test))
best_dt.score(X_train, Y_train)

1.0


1.0

In [55]:
# 보팅 분류
k_model=KNeighborsClassifier()
dt2_model=DecisionTreeClassifier()
lr2_model=LogisticRegression(solver='liblinear')

v_model= VotingClassifier(estimators=[('k_model', k_model), ('dt_model', dt2_model), ('lr_model', lr2_model)],
                         voting='hard')


In [56]:
v_model.fit(X_train.values, Y_train.values)

VotingClassifier(estimators=[('k_model', KNeighborsClassifier()),
                             ('dt_model', DecisionTreeClassifier()),
                             ('lr_model',
                              LogisticRegression(solver='liblinear'))])

In [58]:
# 모델 파라미터 확인
print(f'v_model.classes_: {v_model.classes_}')
print(f'v_model.estimators: {v_model.estimators}')
print(f'v_model.named_estimators: {v_model.named_estimators}')
# print(f'v_model.n_features_in_: {v_model.n_features_in_}')
# print(f'v_model.feature_names_in_: {v_model.feature_names_in_}')

v_model.classes_: [0 1]
v_model.estimators: [('k_model', KNeighborsClassifier()), ('dt_model', DecisionTreeClassifier()), ('lr_model', LogisticRegression(solver='liblinear'))]
v_model.named_estimators: {'k_model': KNeighborsClassifier(), 'dt_model': DecisionTreeClassifier(), 'lr_model': LogisticRegression(solver='liblinear')}


In [59]:
# [4] 성능 확인 => trainDS와 testDS
test_score=v_model.score(X_test.values, Y_test.values)
train_score=v_model.score(X_train.values, Y_train.values)
print(train_score, test_score)

1.0 1.0


----------------------------------------------------------------

In [ ]:
# dt 기반 중요한 features='bruises' ,'gill-size' ,stalk-shape' ,
#               'stalk-surface-below-ring' ,'spore-print-color' ,'population' ,'habitat'

In [70]:
# 피쳐 기반 로지스틱스 회귀
im_features=['bruises' ,'gill-size' ,'stalk-shape' ,'stalk-surface-below-ring' ,'spore-print-color' ,'population' ,'habitat']
# 분리
X_train, X_test, Y_train, Y_test= train_test_split(M_data[im_features], M_data['class'], 
                                                   test_size=.3, stratify=M_data['class'])
print(f'X_train: {X_train.shape},{X_train.ndim}\n\
X_test: {X_test.shape},{X_test.ndim}\n\
Y_train: {Y_train.shape},{Y_train.ndim}\n\
Y_test: {Y_test.shape},{Y_test.ndim}')


X_train: (3950, 7),2
X_test: (1694, 7),2
Y_train: (3950,),1
Y_test: (1694,),1


In [71]:
# 모델 [1]-> 로지스틱스 회귀 with gridsearchCV
# params->'C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 
#           'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'
cv_model= GridSearchCV(estimator=LogisticRegression(solver='liblinear'), param_grid={'C':[1,2,3,5,10]}, 
                       return_train_score=True,
                       verbose=True, scoring=['r2'], cv=5, refit='r2')
cv_model.fit(X_train, Y_train)
result=pd.DataFrame(cv_model.cv_results_)
result
best_lr=cv_model.best_estimator_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


In [72]:
print(best_lr.score(X_test, Y_test))
best_lr.score(X_train,Y_train)
# 정확도가 내려갔지만 train, test간 오차가 줄어듦

0.9344746162927982


0.9359493670886075

In [75]:
# 보팅 분류
k3_model=KNeighborsClassifier()
dt3_model=DecisionTreeClassifier()
lr3_model=LogisticRegression(solver='liblinear')

v_model= VotingClassifier(estimators=[('k_model', k3_model), ('dt_model', dt3_model), ('lr_model', lr3_model)],
                         voting='hard')

v_model.fit(X_train.values, Y_train.values)

VotingClassifier(estimators=[('k_model', KNeighborsClassifier()),
                             ('dt_model', DecisionTreeClassifier()),
                             ('lr_model',
                              LogisticRegression(solver='liblinear'))])

In [76]:
# 모델 파라미터 확인
print(f'v_model.classes_: {v_model.classes_}')
print(f'v_model.estimators: {v_model.estimators}')
print(f'v_model.named_estimators: {v_model.named_estimators}')

# [4] 성능 확인 => trainDS와 testDS
test_score=v_model.score(X_test.values, Y_test.values)
train_score=v_model.score(X_train.values, Y_train.values)
print(train_score, test_score)

v_model.classes_: [0 1]
v_model.estimators: [('k_model', KNeighborsClassifier()), ('dt_model', DecisionTreeClassifier()), ('lr_model', LogisticRegression(solver='liblinear'))]
v_model.named_estimators: {'k_model': KNeighborsClassifier(), 'dt_model': DecisionTreeClassifier(), 'lr_model': LogisticRegression(solver='liblinear')}
1.0 1.0
